In [8]:
import numpy as np
max_pad_len = 7223

def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        #print(sample_rate)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        #print(mfccs.shape)
        
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccs

# Load various imports 

import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'E:/ME SATHIYA/orginaldata/audio/'
metadata = pd.read_csv('E:/ME SATHIYA/orginaldata/metadata/orginaldata.csv')
print(metadata)

features = []


# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'\\'+'caseno'+str(row["caseno"])+'\\'+str(row["file_name"]))
    
    class_label = row["label"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','label'])

print('Finished feature extraction from ', len(featuresdf), 'files')

      file_name  fold  caseno                         label
0        a1.wav     2       5                        Normal
1        6a.wav     2       6                        Normal
2    12(S1).wav     2      12                        Normal
3     12(A).wav     2      12                        Normal
4     12(S).wav     2      12                        Normal
5    12(S1).wav     2      12                        Normal
6     22(A).wav     2      22                        Normal
7    22(A1).wav     2      22                        Normal
8    22(S1).wav     2      22                        Normal
9    22(S1).wav     2      22                        Normal
10    22(W).wav     2      22                        Normal
11    29(A).wav     2      29                        Normal
12   29(A1).wav     2      29                        Normal
13    29(V).wav     2      29                        Normal
14    29(A).wav     2      29                        Normal
15    29(S).wav     2      29           

In [9]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)
print(x_train.shape)

(56, 40, 7223)


In [10]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_rows = 40
num_columns = 7223
num_channels = 1


x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)
num_labels = yy.shape[1]
filter_size = 2
print(x_test.shape)
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_labels, activation='softmax'))

(15, 40, 7223, 1)


In [11]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [12]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 39, 7222, 16)      80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 3611, 16)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 3611, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 3610, 32)      2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 1805, 32)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 9, 1805, 32)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 1804, 64)      

In [15]:
num_epochs = 100
num_batch_size = 32




model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test))


Train on 56 samples, validate on 15 samples
Epoch 1/100
56/56 [==============================] - 9s 153ms/step - loss: 0.2962 - accuracy: 0.8571 - val_loss: 0.2072 - val_accuracy: 0.9333
Epoch 2/100
56/56 [==============================] - 9s 161ms/step - loss: 0.4070 - accuracy: 0.8214 - val_loss: 1.3884 - val_accuracy: 0.6000
Epoch 3/100
56/56 [==============================] - 9s 155ms/step - loss: 0.5621 - accuracy: 0.8036 - val_loss: 1.4813 - val_accuracy: 0.6000
Epoch 4/100
56/56 [==============================] - 8s 143ms/step - loss: 0.4433 - accuracy: 0.8214 - val_loss: 0.3532 - val_accuracy: 0.8667
Epoch 5/100
56/56 [==============================] - 8s 143ms/step - loss: 0.4799 - accuracy: 0.8036 - val_loss: 0.2180 - val_accuracy: 0.8000
Epoch 6/100
56/56 [==============================] - 8s 143ms/step - loss: 0.5257 - accuracy: 0.7143 - val_loss: 0.4306 - val_accuracy: 0.7333
Epoch 7/100
56/56 [==============================] - 8s 143ms/step - loss: 0.3274 - accuracy: 0.85

56/56 [==============================] - 8s 145ms/step - loss: 0.1828 - accuracy: 0.9107 - val_loss: 0.4121 - val_accuracy: 0.7333
Epoch 58/100
56/56 [==============================] - 8s 144ms/step - loss: 0.2063 - accuracy: 0.8750 - val_loss: 0.1799 - val_accuracy: 0.9333
Epoch 59/100
56/56 [==============================] - 8s 146ms/step - loss: 0.2086 - accuracy: 0.8929 - val_loss: 0.1648 - val_accuracy: 0.9333
Epoch 60/100
56/56 [==============================] - 8s 147ms/step - loss: 0.1713 - accuracy: 0.9107 - val_loss: 0.1804 - val_accuracy: 0.9333
Epoch 61/100
56/56 [==============================] - 9s 155ms/step - loss: 0.1888 - accuracy: 0.9107 - val_loss: 0.1970 - val_accuracy: 0.9333
Epoch 62/100
56/56 [==============================] - 9s 160ms/step - loss: 0.1911 - accuracy: 0.9107 - val_loss: 0.1646 - val_accuracy: 0.9333
Epoch 63/100
56/56 [==============================] - 8s 148ms/step - loss: 0.2121 - accuracy: 0.8929 - val_loss: 0.1639 - val_accuracy: 0.9333
Epoch